In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "kiendt/llama3-8b-Psych8k",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
user_input = input()
inputs = tokenizer(
[
    prompt.format(
        "If you are a counsellor, please answer the questions based on the description of the patient.", # instruction
        user_input, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
system_outputs = tokenizer.batch_decode(outputs)
text_str = system_outputs[0]
start_index = text_str.find("Response:\n")
end_index = text_str.find("<|end_of_text|>")

system_output = text_str[start_index+len("Response:\n"):end_index]


Lately, I've been feeling really disconnected from reality. I often find it hard to distinguish between what's real and what's not. For example, last week, I was having dinner with my family, and I suddenly thought my brother was about to attack me. I know it doesn't make any sense, but it felt real in that moment. This has happened a few times, but not all the time. I'm also having a hard time concentrating and remembering things that have happened recently. I've had some depressive episodes in the past, but this seems a bit different to me. Can you help me figure out what's going on?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(system_output)

It sounds like you're experiencing a disconnect between reality and your thoughts, which can be concerning. I'd like to ask you some questions to help me understand this further. Can you please tell me more about what you were doing before you had the thought that your brother was going to attack you? Also, is there anything specific that triggers this feeling of disconnection? Lastly, do you feel like you're able to function normally during these episodes, or are there any additional symptoms you're experiencing?
